In [1]:
import pandas as pd
from scipy.stats import shapiro
import scipy.stats as st
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pingouin as pg

### ANOVA UMA VIA/UM FATOR 

$$H_0: \mu_1 = \mu_2 = \mu_3 ...= \mu_k \text{ (todas as médias são iguais)}\\H_1: \text{Pelo menos uma é diferente das demais} $$

In [2]:
dados_tratamentos = pd.read_csv('Hipertensao.csv', sep= ';', decimal= ',')
dados_tratamentos.head()

,Sujeito,Grupo,Pressao,BC,Triglicerides,Glicemia
0,21,Placebo,175.33,356.33,65.84,93
1,30,Placebo,158.00,378.00,148.15,109
2,27,Placebo,163.67,397.33,115.23,112
3,20,Placebo,184.67,435.33,76.54,113
4,31,Placebo,171.67,365.33,73.25,115


In [3]:
dados_tratamentos.groupby(by='Grupo')['Pressao'].mean().round(2)

Grupo
AH Novo      168.22
AH Padrao    159.40
Placebo      184.64
Name: Pressao, dtype: float64

#### Teste de normalidade dos dados

In [4]:
Placebo = dados_tratamentos.query('Grupo == "Placebo"').Pressao
TratNovo = dados_tratamentos.query('Grupo == "AH Novo"').Pressao
Padrao = dados_tratamentos.query('Grupo == "AH Padrao"').Pressao

In [5]:
alpha = 0.05

stat_test, p_valor = shapiro(Placebo) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 1 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal')
    
stat_test, p_valor = shapiro(TratNovo) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 2 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal')
    
stat_test, p_valor = shapiro(Padrao) 

if(p_valor <= alpha): 
    print('Rejeitamos a Hipótese Nula - Os dados 3 não possuem dist. Normal')
else:
    print('Não rejeitamos a Hipótese Nula - Os dados 3 possuem dist. Normal')

Não rejeitamos a Hipótese Nula - Os dados 1 possuem dist. Normal
Não rejeitamos a Hipótese Nula - Os dados 2 possuem dist. Normal
Não rejeitamos a Hipótese Nula - Os dados 3 possuem dist. Normal


#### Teste de igualdade das variâncias (homogeneidade)

In [6]:
nivel_significancia = 0.05

stat_test, p_valor = st.levene(Placebo, TratNovo, Padrao, center='mean')

print(f"Teste de Levene - Estatística de Teste: {round(stat_test, 2)}, P-valor: {round(p_valor, 4)}")

if p_valor < nivel_significancia:
    print("Rejeita-se a hipótese nula. As variâncias não são iguais.")
else:
    print("Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.")

Teste de Levene - Estatística de Teste: 0.47, P-valor: 0.6323
Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.


In [7]:
stat_test, p_valor = st.bartlett(Placebo, TratNovo, Padrao)

print(f"Teste de Bartlett - Estatística de Teste: {round(stat_test, 2)}, P-valor: {round(p_valor, 4)}")

if p_valor < nivel_significancia:
    print("Rejeita-se a hipótese nula. As variâncias não são iguais.")
else:
    print("Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.")

Teste de Bartlett - Estatística de Teste: 1.58, P-valor: 0.4536
Não há evidências para rejeitar a hipótese nula. As variâncias são iguais.


#### Teste de hipóteses - ANOVA um fator 

In [8]:
alpha = 0.05 

stat_test, p_valor = st.f_oneway(Placebo, TratNovo, Padrao )

# Exibir os resultados de forma mais legível
print(f"Teste Anova de uma via - Estatística de Teste: {round(stat_test, 2)}, P-valor: {round(p_valor, 4)}")
if p_valor < alpha:
    print("Rejeita-se a hipótese nula. Pelo menos uma média é diferente.")
else:
    print("Não há evidências para rejeitar a hipótese nula. As médias são iguais.")

Teste Anova de uma via - Estatística de Teste: 5.52, P-valor: 0.0095
Rejeita-se a hipótese nula. Pelo menos uma média é diferente.


Podemos utilizar o OLS para calcular as informações do teste 

In [9]:
model = ols('Pressao ~ Grupo', data=dados_tratamentos).fit()
anova_table = sm.stats.anova_lm(model, typ=1)
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
Grupo,2.0,3631.035134,1815.517567,5.520544,0.009526
Residual,28.0,9208.239137,328.865683,NaN,NaN


#### Teste Post hoc - detectando quais grupos são diferentes

a) Correção de Tukey 

In [10]:
m_comp = pairwise_tukeyhsd(endog=dados_tratamentos['Pressao'],
                           groups=dados_tratamentos['Grupo'], alpha=alpha)
print(m_comp)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower    upper  reject
-----------------------------------------------------------
  AH Novo AH Padrao  -8.8232 0.5469 -29.4402 11.7938  False
  AH Novo   Placebo  16.4169 0.1183  -3.3695 36.2034  False
AH Padrao   Placebo  25.2402 0.0082   6.0273  44.453   True
-----------------------------------------------------------


Verificamos que existe diferença entre os tratamentos, ou seja há efeito do tratamento sobre a pressão arterial. O post-hoc do teste de Tukey hsd mostrou que existe diferença entre o **Placebo** e entre o grupo de anti hipertensivo padrão **AH Padrao**.

b) Correção de Bonferroni 

In [11]:
def testes_t_pareados_bonferroni(df, coluna_grupo, coluna_valor, alfa = 0.05):
    from itertools import combinations

    # Cria uma lista única de grupos
    grupos = df[coluna_grupo].unique()
    testes_t = []
    
    # Gera todas as combinações únicas de pares de grupos
    combinacoes = combinations(grupos, 2)

    # Itera pela lista de grupos
    for grupo1, grupo2 in combinacoes:
        tipo_1 = df[df[coluna_grupo] == grupo1]
        tipo_2 = df[df[coluna_grupo] == grupo2]

        t, p = st.ttest_ind(tipo_1[coluna_valor], tipo_2[coluna_valor])

        # Adiciona cada resultado de teste t a uma lista de pares t, p
        testes_t.append([grupo1 + ' - ' + grupo2, t.round(4), p.round(4)])

        # Imprime os resultados
        print(f'Resultados teste t: {grupo1}/{grupo2} - t: {round(t, 2)}, p: {round(p, 4)}')
        
    # O novo limiar para significância estatística = 0.05 / comprimento da lista de testes t
    alfa_ajustado = alfa/ len(testes_t)
    print(f'\n P-valor do teste t abaixo do alfa ajustado: {round(alfa_ajustado,4)}')

    # Imprime apenas os resultados qaue são significativos com base no novo limiar
    for par in testes_t:
        if par[2] <= alfa_ajustado:  # O valor p está no segundo índice, por isso é 2
            print(par)

In [12]:
testes_t_pareados_bonferroni(dados_tratamentos, coluna_grupo='Grupo', coluna_valor='Pressao')

Resultados teste t: Placebo/AH Novo - t: 2.17, p: 0.0427
Resultados teste t: Placebo/AH Padrao - t: 2.98, p: 0.0074
Resultados teste t: AH Novo/AH Padrao - t: 1.12, p: 0.279

 P-valor do teste t abaixo do alfa ajustado: 0.0167
['Placebo - AH Padrao', 2.9813, 0.0074]


### ANOVA UMA VIA/UM FATOR - com medidas repetidas 

 $$ H_0: \mu_1 = \mu_2 = \mu_3  = \mu_4\text{ (todas as médias são iguais)}\\H_1: \text{Pelo menos uma é diferente das demais} $$

In [13]:
notas_alunos = pd.read_csv('Notas_alunos.csv', sep= ';', decimal= ',')
notas_alunos.head(5)

df = pd.melt(notas_alunos, id_vars='Aluno', var_name='Professor', value_name='Nota')
df.head()

,Aluno,Professor,Nota
0,1,Prof1,6.2
1,2,Prof1,7.8
2,3,Prof1,8.1
3,4,Prof1,9.0
4,5,Prof1,6.0


#### Teste de normalidade dos dados usando o pacote pingouin

In [14]:
alpha = 0.05
pg.normality(data=df, dv='Nota', group='Professor', alpha = alpha)

,W,pval,normal
Professor,,,
Prof1,0.931535,0.053943,True
Prof2,0.937347,0.077172,True
Prof3,0.935625,0.069383,True
Prof4,0.962681,0.362035,True


#### Teste de esfericidade dos dados usando o pacote pingouin

Esfericidade é a condição em que as variâncias das diferenças entre todas as combinações de grupos (níveis) relacionados são iguais. A violação da esfericidade ocorre quando as variações das diferenças entre todas as combinações de grupos relacionados não são iguais.

Ref: https://pingouin-stats.org/build/html/generated/pingouin.sphericity.html

In [15]:
spher, _, chisq, dof, pval =  pg.sphericity(data=df, dv='Nota', subject='Aluno', within=['Professor'], 
                                            alpha = alpha)


print(f"Teste de Mauchly para Esfericidade:")
print(f"Esfericidade Atingida: {spher}")
print(f"Estatística Qui-Quadrado: {round(chisq, 3)}")
print(f"Graus de Liberdade: {dof}")
print(f"Valor-p: {round(pval, 3)}")

# Decidir sobre a esfericidade com base no valor-p
if pval < alpha:
    print("Rejeitar H0: Esfericidade Não Atingida")
else:
    print("Não rejeitar H0: Esfericidade Atingida")

Teste de Mauchly para Esfericidade:
Esfericidade Atingida: False
Estatística Qui-Quadrado: 28.346
Graus de Liberdade: 5
Valor-p: 0.0
Rejeitar H0: Esfericidade Não Atingida


#### Correção para esfericidade 

m cenários onde o valor p é inferior a 0,05 e rejeitamos a hipótese nula do teste de esfericidade de Mauchly, normalmente aplicamos uma correção aos graus de liberdade usados para calcular o índice F.

Existem três correções que podemos aplicar:

* Huynh-Feldt (menos conservador)
* Greenhouse-Geisser
* Lower-bound (mais conservador)

Cada uma dessas correções tende a aumentar os valores de p na tabela de resultados da ANOVA de medidas repetidas para explicar o fato de que a suposição de esfericidade é violada.

Ref: https://pingouin-stats.org/build/html/generated/pingouin.rm_anova.html#pingouin.rm_anova

In [16]:
aov = pg.rm_anova(dv='Nota', within='Professor', subject='Aluno', data=df, detailed = False)
pg.print_table(aov)


ANOVA SUMMARY

Source       ddof1    ddof2       F    p-unc    p-GG-corr    ng2    eps  sphericity      W-spher    p-spher
---------  -------  -------  ------  -------  -----------  -----  -----  ------------  ---------  ---------
Professor        3       87  22.337    0.000        0.000  0.047  0.615  False             0.360      0.000



In [17]:
# 'Source': Nome do fator dentro do grupo
# 'ddof1': Graus de liberdade (numerador)
# 'ddof2': Graus de liberdade (denominador)
# 'F': Valor F
# 'p-unc': Valor p não corrigido
# 'ng2': Tamanho do efeito eta-quadrado generalizado
# 'eps': Fator épsilon de Greenhouse-Geisser (= índice de esfericidade)
# 'p-GG-corr': Valor p corrigido por Greenhouse-Geisser
# 'W-spher': Estatística de teste de esfericidade
# 'p-spher': valor p do teste de esfericidade
# 'sphericity': esfericidade dos dados (booleano)

In [18]:
spher, _, chisq, dof, pval =  pg.sphericity(data=df, dv='Nota', subject='Aluno', within=['Professor'], 
                                            alpha = alpha)

if pval < alpha:
    print("Rejeitar H0: Esfericidade Não Atingida")
else:
    print("Não rejeitar H0: Esfericidade Atingida")

Rejeitar H0: Esfericidade Não Atingida


#### Teste Post hoc - detectando quais grupos são diferentes

Ref: https://pingouin-stats.org/build/html/generated/pingouin.pairwise_tests.html

a) Correção de Bonferroni 

In [19]:
posthocs = pg.pairwise_tests(dv='Nota', within='Professor', subject='Aluno', data=df ,
                             padjust = 'bonf', alpha = alpha)
posthocs

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Professor,Prof1,Prof2,True,True,4.147334,29.0,two-sided,0.000268,0.001607,bonf,106.985,0.190219
1,Professor,Prof1,Prof3,True,True,3.209474,29.0,two-sided,0.003239,0.019434,bonf,11.841,0.196835
2,Professor,Prof1,Prof4,True,True,5.812494,29.0,two-sided,0.000003,0.000016,bonf,7120.858,0.571887
3,Professor,Prof2,Prof3,True,True,0.117449,29.0,two-sided,0.907314,1.000000,bonf,0.196,0.005671
4,Professor,Prof2,Prof4,True,True,4.725388,29.0,two-sided,0.000054,0.000326,bonf,450.439,0.402433
5,Professor,Prof3,Prof4,True,True,4.537230,29.0,two-sided,0.000092,0.000550,bonf,280.933,0.399543


b) Correção de Sidak

In [20]:
posthocs = pg.pairwise_tests(dv='Nota', within='Professor', subject='Aluno', data=df ,
                             padjust = 'sidak', alpha = alpha)

posthocs

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Professor,Prof1,Prof2,True,True,4.147334,29.0,two-sided,0.000268,0.001606,sidak,106.985,0.190219
1,Professor,Prof1,Prof3,True,True,3.209474,29.0,two-sided,0.003239,0.019278,sidak,11.841,0.196835
2,Professor,Prof1,Prof4,True,True,5.812494,29.0,two-sided,0.000003,0.000016,sidak,7120.858,0.571887
3,Professor,Prof2,Prof3,True,True,0.117449,29.0,two-sided,0.907314,0.999999,sidak,0.196,0.005671
4,Professor,Prof2,Prof4,True,True,4.725388,29.0,two-sided,0.000054,0.000326,sidak,450.439,0.402433
5,Professor,Prof3,Prof4,True,True,4.537230,29.0,two-sided,0.000092,0.000549,sidak,280.933,0.399543


c) Correção de Tukey

In [21]:
m_comp = pairwise_tukeyhsd(endog=df['Nota'],
                           groups=df['Professor'], alpha=alpha)
print(m_comp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
 Prof1  Prof2  -0.3533 0.8732  -1.569 0.8623  False
 Prof1  Prof3  -0.3633 0.8638  -1.579 0.8523  False
 Prof1  Prof4  -1.0833 0.0987  -2.299 0.1323  False
 Prof2  Prof3    -0.01    1.0 -1.2257 1.2057  False
 Prof2  Prof4    -0.73 0.4025 -1.9457 0.4857  False
 Prof3  Prof4    -0.72 0.4149 -1.9357 0.4957  False
---------------------------------------------------
